## Different imports and setup

In [ ]:
# Standard import
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
# Machine learning libraries
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Nadam, Adadelta
from tensorflow.keras.losses import MeanSquaredError

from tqdm.keras import TqdmCallback
from tqdm.notebook import tqdm

In [ ]:
# Custom plots file and tensorflow models
from Tools import Plot
from Tools import Custom_models
from Tools import Helpers

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('Not working on GPU !')
else:
  print('Found GPU at: {}'.format(device_name))

In [ ]:
tf.config.list_physical_devices()

## Set the seeds and extract the training data

In [ ]:
# Set the seed for reproducibility
seed = 6
np.random.seed(seed)
tf.random.set_seed(seed*2)

In [ ]:
# Load the data
features_train, labels_train, features_val, labels_val, features_test, labels_test = Helpers.Load_and_split_val(
    'Data/tanh_10D.txt', D=10, seed=seed, ratio_test=0.1, ratio_val=0.1)

In [ ]:
print(f'We will be training on {len(features_train)} examples !')
print(f'We will be validating on {len(features_val)} examples !')
print(f'And testing on {len(features_test)} examples !')

In [ ]:
features_train.shape

## Create and train the model

In [ ]:
# Create the model
learning_rate = 5e-5
batch = 32
K = 100
#Ks = [64, 1024]
depth = 2
K_out = 1
N_epochs = 250

N_average = 1

histories = []

In [ ]:
# Train

for i in tqdm(range(N_average)):
        
    save_name = f'Saved_models/10D/Model_sum/K_{K}-depth_{depth}'
            
    name = f'2_layers-batch_{batch}-lr_{learning_rate:.1e}'
    opt = Adam(learning_rate=learning_rate)
    
    model = Custom_models.Model_depth(K1=K, K_output=K_out, depth=depth, name=name)
    #model = Custom_models.Model_depth_Ks(Ks=Ks, K_output=K_out, name=name)
    model.compile(loss=MeanSquaredError(), optimizer=opt)
        
    #logger = tf.keras.callbacks.CSVLogger(save_name + '.csv')
    #saver = tf.keras.callbacks.ModelCheckpoint(save_name, save_best_only=True, mode='min')

    history = model.fit(features_train, labels_train, validation_data=(features_val, labels_val),
                        epochs=N_epochs, batch_size=batch, validation_batch_size=batch, verbose=0,
                        shuffle=True, initial_epoch=0, callbacks=[TqdmCallback()])#, logger, saver])
    
    histories.append(history)

## Check the results

In [ ]:
MSE = []
MAE = []
MaAE = []

for i in range(N_average):
    model_name = f'Saved_models/1D/1_layer_study/K_{K}-iteration_{i}'
    model = keras.models.load_model(model_name)
    
    MSE.append(Helpers.MSE(model, features_test, labels_test))
    MAE.append(Helpers.MAE(model, features_test, labels_test))
    MaAE.append(Helpers.MSE(model, features_test, labels_test))

In [ ]:
with np.printoptions(formatter={'all':lambda x: f'{x:.4e}'}):
    print(MSE)
    print('\n')
    print(MAE)
    print('\n')
    print(MaAE)

In [ ]:
mean_mse = np.mean(MSE)
mean_mae = np.mean(MAE)
mean_maae = np.mean(MaAE)

print(f'MSE : min : {min(MSE):.4e} // mean : {mean_mse:.4e} // max : {max(MSE):.4e} \n')
print(f'MAE : min : {min(MAE):.4e} // mean : {mean_mae:.4e} // max : {max(MAE):.4e} \n')
print(f'MaAE : min : {min(MaAE):.4e} // mean : {mean_maae:.4e} // max : {max(MaAE):.4e} \n')

In [ ]:
print(np.argmin(MSE))
print(np.argmin(MAE))
print(np.argmin(MaAE))

## Check the efficiency of training

In [ ]:
filename = 'Figures/10D/k_50.pdf'
save = False

history = histories[0]

plt.figure()
plt.semilogy(history.epoch, np.array(history.history['loss']), label='Training')
plt.semilogy(history.epoch, np.array(history.history['val_loss']), label = 'Validation')
plt.xlabel('Epoch')
plt.ylabel('Mean squared error')
plt.legend()
plt.grid()
if (save):
    plt.savefig(filename, bbox_inches='tight')
plt.show()

## Check on a grid

In [ ]:
x = np.linspace(0, 1, 50000)
coord = np.zeros((len(x), 10))
for i in range(10):
    coord[:,i] = x
    
center = np.zeros(10)
r = np.sqrt(10)/2       
k = 50

rad = np.linalg.norm(coord - center, ord=2, axis=1)
pred = model.predict(coord).ravel()

y_true = 1/2*(1 + np.tanh(-k*(rad - r)))

In [ ]:
save = False
name = 'Figures/10D/2_layers/approximation_other.pdf'

plt.figure()
plt.plot(rad, y_true, 'b-', label='True')
plt.plot(rad, pred, 'r--', label='Prediction on the\nline $x_1 = ... = x_{10}$')
plt.xlabel('$r$')
plt.ylabel('$f(r)$')
plt.legend()
plt.grid()
if (save):
    plt.savefig(name, bbox_inches='tight')
plt.show()